In [1]:
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import mne
import os
from mne import viz
from mne import io
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
import zipfile
import sys
import pandas as pd
import numpy as np
from IPython.utils import io

## Participant Selection

In [ ]:
# Listing participant files
folder_names = []
participants = sorted(os.listdir('./sourcedata'))
for p in participants:
    print(f'{p}:')
    conditions = os.listdir(f'./sourcedata/{p}/eeg')
    for c in conditions:
        print(f'    -{c}')
        folder_names.append(c)
        file_names = os.listdir(f'./sourcedata/{p}/eeg/{c}')
        for fname in file_names:
            print(f'       <{fname}>\n')

In [3]:
#------select-file------#
project = 'derivative_project00'
ID = 'MSICU07'
task = 'RS_ST_Taken_Tennis'
#-----------------------#


source_path = f'./sourcedata/sub-{ID}/eeg/sub-{ID}_task-{task}_eeg' 
files = os.listdir(source_path)
for file_name in files:
    parts = file_name.split('.')
    fname = parts[0]
    print(fname)

Session 20220607 1636


## Data

In [4]:
#Creating a preproc directory from sourcefile
print('(1) Searching for working directorys.')
if 'HC' in ID:
    new_path = f'./{project}/HC/sub-{ID}/eeg/sub-{ID}_task-{task}_eeg'
else:
    new_path = f'./{project}/PT/sub-{ID}/eeg/sub-{ID}_task-{task}_eeg'
if not os.path.exists(f'./{new_path}'):
    os.makedirs(f'{new_path}')
    print('    - No directory found. New directory made.')
else:
    print('    - This directory already exists. \n    - Continue with existing file.') 

#unzip source files to the participants working directory
print('\n(2) Unzipping files into prepoc directory.')         
if not os.path.exists(f'./{new_path}/{fname}'):
    with zipfile.ZipFile(f'{source_path}/{fname}.zip', 'r') as zip_ref:
        zip_ref.extractall(new_path)
        print (f'    - sub-{ID}_task-{task} sourcefile was unzipped to the participants working directory.')
else:
    print(f'    - Cannot unzip files because {fname} already exists.\n    - Continue with existing file.')

#Renaming the file       
if not os.path.exists(f'./{new_path}/{fname}.mff'):
    os.rename(f'{new_path}/{fname}', f'{new_path}/{fname}.mff')
    print(f'    - sub-{ID}_task-{task}_eeg/{fname}.mff was renamed with .mff suffix')
else:
    print(f'    - cant rename file to .mff because {fname}.mff file already exisits. \n    - Continuing with existing file')

#Loading metadata
print(f'\n(3) loading metadata and downsampling for {ID}_{task}')
data_path = f'{new_path}/{fname}.mff'
raw = mne.io.read_raw_egi(data_path)
raw

#Downsampling to 250Hz
data = raw.copy()
raw_sfreq = raw.info['sfreq']
data_sfreq = data.info['sfreq']
print(f'    Downsampling...\n    - The raw sampling frequincy is {raw_sfreq}Hz.')
if data_sfreq == 250:
    print('    - Cannot downsample because data is already sampled at 250Hz.')
elif data_sfreq > 250:
    data.resample(250)
    print('    - Data was downsampled to 250Hz.')
else:
    print(f'    - Error: Data sampling frequency is {raw_sfreq}Hz.')
    answer = input('      - Do you want to continue?:')
    if answer.lower().startswith("y"):
        print("      - ok, carry on then")
    elif answer.lower().startswith("n"):
        print("      - ok, sayonnara")
        sys.exit()
        
#loading in data      
print('\n(4) Loading data into memory...')
data.load_data() 


(1) Searching for working directorys.
    - No directory found. New directory made.

(2) Unzipping files into prepoc directory.
    - sub-MSICU07_task-RS_ST_Taken_Tennis sourcefile was unzipped to the participants working directory.
    - sub-MSICU07_task-RS_ST_Taken_Tennis_eeg/Session 20220607 1636.mff was renamed with .mff suffix

(3) loading metadata and downsampling for MSICU07_RS_ST_Taken_Tennis
Reading EGI MFF Header from /imaging/owenlab/EGI_ICU/derivative_project00/PT/sub-MSICU07/eeg/sub-MSICU07_task-RS_ST_Taken_Tennis_eeg/Session 20220607 1636.mff...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {sync} ...
    Downsampling...
    - The raw sampling frequincy is 250.0Hz.
    - Cannot downsample because data is already sampled at 250Hz.

(4) Loading data into memory...
Reading 0 ... 496769  =      0.000 ...  1987.076 secs...


<RawMff | signal1.bin, 135 x 496770 (1987.1 s), ~511.8 MB, data loaded>

## Inital Events & Initial Crop
redefine trigger events after cropping

In [6]:
data.plot()

Using matplotlib as 2D backend.


<MNEBrowseFigure size 800x800 with 4 Axes>

Channels marked as bad:
none


In [5]:
# Timeing
print('(1) Data timing...\n    -', data)
print(f'    - time(s): {data.tmin} - {data.tmax}')
print(f'    - samples: {data.first_samp} - {data.last_samp}')
seconds = data.tmax
seconds = seconds % (24 * 3600)
hour = seconds // 3600
seconds %= 3600
minutes = seconds // 60
seconds %= 60
print("    -* Duration = %dhr : %02dmin : %02dsec" % (hour, minutes, seconds))

# Trigger Channels
print('\n(2) Trigger Channels')
triggers = []
for i in data.ch_names:
    if not 'E' in i:
        if not 'sync' in i:
            if not 'STI' in i:
                if not 'V' in i:
                    print ("    - Trigger ch.",i)
                    triggers.append(i)

# Initial events
print('\n(3) Initial events')
initial_events = mne.find_events(data, stim_channel='STI 014', initial_event=True)
for i in range(initial_events.shape[0]):
    exec('initial_events_'+str(i+1) +'='+ str(initial_events[i,0]/data.info['sfreq']))
    exec("print('    - initial_events_' + str(i+1)+ ':', initial_events_" + str(i+1)+",'s.')")


# (4) crop

# (5) Mark Triggers

(1) Data timing...
    - <RawMff | signal1.bin, 135 x 496770 (1987.1 s), ~511.8 MB, data loaded>
    - time(s): 0.0 - 1987.076
    - samples: 0 - 496769
    -* Duration = 0hr : 33min : 07sec

(2) Trigger Channels
    - Trigger ch. 9   
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 20  

(3) Initial events
4 events found
Event IDs: [1 2 3 4]
    - initial_events_1: 6.232 s.
    - initial_events_2: 600.236 s.
    - initial_events_3: 1000.396 s.
    - initial_events_4: 1708.172 s.


In [6]:
data.crop(tmin = initial_events_1) #tmax = 380)
#data.crop(tmin = initial_events_1, tmax = initial_events_6)
start_time = data.first_samp/data.info['sfreq']


print('(1) Data timing...\n    -', data)
print(f'    - time(s): {data.tmin} - {data.tmax}')
print(f'    - samples: {data.first_samp} - {data.last_samp}')
seconds = data.tmax
seconds = seconds % (24 * 3600)
hour = seconds // 3600
seconds %= 3600
minutes = seconds // 60
seconds %= 60
print("    -* Duration = %dhr : %02dmin : %02dsec" % (hour, minutes, seconds))

### Defining trigger times

In [7]:
#Assigning variables for event onset times
#print('\n(4) Assigning variables for event onset times:')
print('\n______________________________________________\n--------------Trigger Channels_2--------------')
for i in triggers:
    print ("\n______________________________________________\nCh.", i)
    if '09' in i: 
        with io.capture_output() as captured:
            trig09=(mne.find_events(data, stim_channel='09  ', initial_event=True)[0, 0]/250) - (start_time)
        print ("    > Rest Start (s):", trig09, "<")
    elif '19' in i: 
        with io.capture_output() as captured:
            trig19=(mne.find_events(data, stim_channel='19  ', initial_event=True)[0, 0]/250) - (start_time)
        print ("    > Rest End (s):", trig19, "<")
    elif '12' in i: 
        with io.capture_output() as captured:
            trig12=(mne.find_events(data, stim_channel='12  ', initial_event=True)[0, 0]/250) - (start_time)
        print ("    > Scrambled Start (s):", trig12, "<")
    elif '22' in i: 
        with io.capture_output() as captured:
            trig22=(mne.find_events(data, stim_channel='22  ', initial_event=True)[0, 0]/250) - (start_time)
        print ("    > Scrambled End (s):", trig22, "<")
    elif '10' in i:
        with io.capture_output() as captured:
            trig10=(mne.find_events(data, stim_channel='10  ', initial_event=True)[0, 0]/250) - (start_time)
        print ("    > Taken Start (s):", trig10, "<")
    elif '20' in i: 
        with io.capture_output() as captured:
            trig20=(mne.find_events(data, stim_channel='20  ', initial_event=True)[0, 0]/250) - (start_time)
        print ("    > Taken End (s):", trig20, "<")
    #ERROR: other triggers found
    elif '11' in i:
        with io.capture_output() as captured:
            trig11=(mne.find_events(data, stim_channel='11  ', initial_event=True)[0, 0]/250) - (start_time)
        print ("    > ERROR idk what trigger 11 is:", trig11, "< (for HC, this was a second viewing of intact taken)")
    elif '99' in i:
        with io.capture_output() as captured:
            trig99=(mne.find_events(data, stim_channel='99  ', initial_event=True)[0, 0]/250) - (start_time)
        print ("    > ERROR idk what trigger 99 is:", trig99, "<")        
    else:
        print (f'\n____________( {i} )____________\nERROR: idk what {i} is!')


______________________________________________
--------------Trigger Channels_2--------------

______________________________________________
Ch. 12  
    > Scrambled Start (s): 0.0 <

______________________________________________
Ch. 10  
    > Taken Start (s): 631.808 <


#### Power Spectral Density (PSD)

In [8]:
print('Number of channels in EEG:', len(data.ch_names))
print ('\nAll channel names:', data.ch_names)
print('\nData timing:', data)

Number of channels in EEG: 133

All channel names: ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19', 'E20', 'E21', 'E22', 'E23', 'E24', 'E25', 'E26', 'E27', 'E28', 'E29', 'E30', 'E31', 'E32', 'E33', 'E34', 'E35', 'E36', 'E37', 'E38', 'E39', 'E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46', 'E47', 'E48', 'E49', 'E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64', 'E65', 'E66', 'E67', 'E68', 'E69', 'E70', 'E71', 'E72', 'E73', 'E74', 'E75', 'E76', 'E77', 'E78', 'E79', 'E80', 'E81', 'E82', 'E83', 'E84', 'E85', 'E86', 'E87', 'E88', 'E89', 'E90', 'E91', 'E92', 'E93', 'E94', 'E95', 'E96', 'E97', 'E98', 'E99', 'E100', 'E101', 'E102', 'E103', 'E104', 'E105', 'E106', 'E107', 'E108', 'E109', 'E110', 'E111', 'E112', 'E113', 'E114', 'E115', 'E116', 'E117', 'E118', 'E119', 'E120', 'E121', 'E122', 'E123', 'E124', 'E125', 'E126', 'E127', 'E128', 'Vertex Reference', 'sync', '12 

In [9]:
#1st PDS
viz.plot_raw_psd(data, picks='eeg', exclude= ['Vertex Reference'],fmax=70)

NOTE: plot_raw_psd() is a legacy function. New code should use Raw.compute_psd().plot().
NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Setting 37750 of 259999 (14.52%) samples to NaN, retaining 222249 (85.48%) samples.
Effective window size : 8.192 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s finished


[]

In [10]:
data.plot(duration = 30, n_channels=50)

Using matplotlib as 2D backend.


<MNEBrowseFigure size 1176x800 with 4 Axes>

Channels marked as bad:
none


## Filter

In [ ]:
data_filter = data.copy()
data_filter.filter(0.1, 50)
data_filter.notch_filter(60)
#data_filter.notch_filter(50)
data_filter.compute_psd(picks='eeg', exclude=['Vertex Reference'], fmax=70).plot()

## Remove Bad Electrodes

In [12]:
data_filters_bads = data_filter.copy()#.pick_types(eeg=True)
print('Number of channels in EEG:', len(data_filters_bads.ch_names))
print('Bad Electrodes:', data_filters_bads.info['bads'])
print ('All channel names:', data_filters_bads.ch_names)

Number of channels in EEG:
133
Bad Electrodes:
[]
all channel names
['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19', 'E20', 'E21', 'E22', 'E23', 'E24', 'E25', 'E26', 'E27', 'E28', 'E29', 'E30', 'E31', 'E32', 'E33', 'E34', 'E35', 'E36', 'E37', 'E38', 'E39', 'E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46', 'E47', 'E48', 'E49', 'E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64', 'E65', 'E66', 'E67', 'E68', 'E69', 'E70', 'E71', 'E72', 'E73', 'E74', 'E75', 'E76', 'E77', 'E78', 'E79', 'E80', 'E81', 'E82', 'E83', 'E84', 'E85', 'E86', 'E87', 'E88', 'E89', 'E90', 'E91', 'E92', 'E93', 'E94', 'E95', 'E96', 'E97', 'E98', 'E99', 'E100', 'E101', 'E102', 'E103', 'E104', 'E105', 'E106', 'E107', 'E108', 'E109', 'E110', 'E111', 'E112', 'E113', 'E114', 'E115', 'E116', 'E117', 'E118', 'E119', 'E120', 'E121', 'E122', 'E123', 'E124', 'E125', 'E126', 'E127', 'E128', 'Vertex Referen

In [15]:
### Removing Bad Electrodes ###

# AUTOMATIC detecion of bad channels
auto_annot, auto_bads = mne.preprocessing.annotate_amplitude(data_filters_bads, 
                    peak=10e-6,
                    flat=0.01e-6 ,
                    bad_percent=5,
                    min_duration=0.005)
print(f'automatic bads: {auto_bads}')
print(len(auto_bads))
print(f'anotated: {auto_annot}')
print(len(auto_annot))
data_filter_bads.info['bads'].extend(auto_bads)
print(data_filter_bads.info['bads'])

# plot & MANUAL detection of bads
#data_filter_bads.info['bads'].extend(['E1', 'E56', 'E49', 'E78', 'E79', 'E114', 'E120', 'E121', 'E125', 'E128', 'E44', 'E54', 'E55', 'E61', 'E62'])
viz.plot_raw_psd(data_filter_bads, picks='eeg', exclude=['Vertex Reference'], fmax=70)
data_filter_bads.plot(duration = 10, n_channels=50) 
marked_bad = data_filter_bads.info['bads']

Finding segments below or above PTP threshold.
automatic bads: ['E49', 'Vertex Reference', 'E1', 'E2', 'E3', 'E8', 'E14', 'E15', 'E17', 'E21', 'E22', 'E25', 'E26', 'E27', 'E32', 'E33', 'E34', 'E38', 'E39', 'E40', 'E43', 'E44', 'E45', 'E46', 'E48', 'E49', 'E50', 'E53', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E63', 'E64', 'E65', 'E66', 'E69', 'E70', 'E73', 'E74', 'E75', 'E76', 'E81', 'E82', 'E83', 'E84', 'E88', 'E89', 'E90', 'E91', 'E95', 'E96', 'E97', 'E99', 'E100', 'E101', 'E102', 'E107', 'E108', 'E109', 'E113', 'E114', 'E115', 'E116', 'E119', 'E120', 'E121', 'E122', 'E123', 'E125', 'E126', 'E127', 'E128']
75
anotated: <Annotations | 65858 segments: BAD_flat (41015), BAD_peak (24843)>
65858


### - Interpolate (optional)

In [ ]:
### Interpolate ###
test_interpolate = data_filter_bads.copy()
# test_interpolate.interpolate_bads(reset_bads=True, mode='accurate')
data_interpolated = mne.preprocessing.interpolate_bad_channels(test_interpolate, marked_bad)

### - Annotate time series (optional)

In [5]:
### Auto-annotate ###
auto_annotations = mne.preprocessing.annotate_amplitude(
    data_filter_bads, 
    peak=100e-6, 
    flat=1e-6, 
    bad_percent=5, 
    min_duration=0.005
print(f'Automatic annotations: {auto_annotations}')
print(len(auto_annotations))
# Apply annotations to the data
test_annotate = data_filter_bads.copy()
test_annotate.set_annotations(auto_annotations)
# Print details of the annotations
print(test_annotate.annotations.orig_time)
print("Number of annodations made:", len(test_annotate.annotations))
print("    - Annotation names:", test_annotate.annotations.description)
print("    - Start times:", set(test_annotate.annotations.onset))
print("    - Durations:", test_annotate.annotations.duration)
# Save and read annotations
print('\nSaving .txt')
test_annotate.annotations.save(f'{new_path}/sub-{ID}_task-{task}_saved-annotations.txt', overwrite=True)
annot_from_file = mne.read_annotations(f'{new_path}/sub-{ID}_task-{task}_saved-annotations.txt')
print("    - ", annot_from_file)

#remove annotated parts 

2016-05-20 17:44:59.212000+00:00
Number of annodations made: 2
    - Annotation names: ['BAD_ACQ_SKIP' 'BAD_ACQ_SKIP']
    - Start times: {560.09, 971.182}
    - Durations: [73.048 77.952]

Saving .txt
Overwriting existing file.
    -  <Annotations | 2 segments: BAD_ACQ_SKIP (2)>


## Rerefrence

In [ ]:
### re-refrence ###
data_filter_bads_reref = data_filter_bads.copy()
data_filter_bads_reref.set_eeg_reference(ref_channels='average')


In [ ]:
data_filter_bads_reref.compute_psd(picks='eeg', fmax=70).plot()

In [ ]:
data_filter_bads_reref.plot()

## Delete nonbrain electrods and bads

In [ ]:
#Remove non-brain Electrodes
data_filter_bads_reref_brainonly = data_filter_bads_reref.copy().pick_types(eeg=True)

non_brain_el = ['E127', 'E126', 'E17', 'E21', 'E14', 'E25', 'E8', 'E128', 
                'E125', 'E43', 'E120', 'E48', 'E119', 'E49', 'E113', 'E81', 
                'E73', 'E88', 'E68', 'E94', 'E63', 'E99', 'E56', 'E107' ]

#only add non-brain channels if not already part of noisy channels
for e in non_brain_el: 
    if e in data_filter_bads_reref_brainonly.info['ch_names']:
        if e not in marked_bad:
            if e not in data_filter_bads_reref_brainonly.info['bads']:
                ddata_filter_bads_reref_brainonly.info['bads'].append(e)
print(data_filter_bads_reref_brainonly.info['bads'])

#remove bads
data_filter_bads_reref_brainonly.drop_channels(data_filter_bads_reref_brainonly.info['bads'])

#### - Interpolate

In [ ]:
data_filter_bads_reref_brainonly_interpolate = data_filter_bads_reref_brainonly.copy()


In [ ]:
picks = mne.pick_channels_regexp(raw.ch_names, regexp='EEG 05.')
raw.plot(order=picks, n_channels=len(picks))

In [ ]:
eeg_data = raw.copy().pick_types(meg=False, eeg=True, exclude=[])
eeg_data_interp = eeg_data.copy().interpolate_bads(reset_bads=False)

for title, data in zip(['orig.', 'interp.'], [eeg_data, eeg_data_interp]):
    with mne.viz.use_browser_backend('matplotlib'):
        fig = data.plot(butterfly=True, color='#00000022', bad_color='r')
    fig.subplots_adjust(top=0.9)
    fig.suptitle(title, size='xx-large', weight='bold')

# Crop data by stimulus

In [ ]:
#Splitting data into events
data_filter_bads_reref_brainonly_interpolate.plot()

In [ ]:
#Detect the trigger events
events = mne.find_events(data_filter_bads_reref_brainonly_interpolate) #stim_channel='STI 014', initial_event=True)
print(events)
#number of eventss
num_events = events.shape[0]
print("Number of events:", num_events)

#######
event_onsets = events[:,0] / data.info['sfreq']
for i in range(len(event_onsets)):
    exec("event_" + str(i+1) + " = " + str(event_onsets[i]))
for i in range(len(event_onsets)):
    exec("print('event_' + str(i+1), 'onset:', event_" + str(i+1) + ")")

In [ ]:

# Crop (choose the event times for the associated conditions)
if num_events == 6:
    print(f'all data\n  - {data.time_as_index}\n  - {data.first_samp}\n  - {data.last_samp}\n  - {data.tmin}\n  - {data.tmax}')
    
    rest_preproc = data_filter_bads_reref_brainonly_interpolate.copy().crop(tmin=event_onset_1, tmax=event_onset_2)
    scrambled_preproc = data_filter_bads_reref_brainonly_interpolate.copy().crop(tmin=event_onset_3, tmax=event_onset_4)
    taken_preproc = data_filter_bads_reref_brainonly_interpolate.copy().crop(tmin=event_onset_5, tmax=event_onset_6)
    
    print(f'rest\n  - {rest_preproc.time_as_index}\n  - {rest_preproc.first_samp}\n  - {rest_preproc.last_samp}\n  - {rest_preproc.tmin}\n  - {rest_preproc.tmax}')
    print(f'scrambled\n  - {scrambled_preproc.time_as_index}\n  - {scrambled_preproc.first_samp}\n  - {scrambled_preproc.last_samp}\n  - {scrambled_preproc.tmin}\n  - {scrambled_preproc.tmax}')
    print(f'taken\n  - {taken_preproc.time_as_index}\n  - {taken_preproc.first_samp}\n  - {taken_preproc.last_samp}\n  - {taken_preproc.tmin}\n  - {taken_preproc.tmax}')
else:
    print(f'ERROR: You only have {num_events} events, check what events are missing')


### Save individual files (checkpoint)

In [ ]:
if os.path.isdir(f'/{project}/preproc_results')==False:
    os.mkdir(f'{project}/preproc_results')
if not os.path.exists(f'./{project}/preproc_results/sub-{ID}/sub-{ID}_task-{task}'):
    os.makedirs(f'./{project}/preproc_results/sub-{ID}/sub-{ID}_task-{task}')
    print('new directory made')
data_filter_bads_reref_brainonly_interpolate.save(f'./{project}/preproc_results/sub-{ID}/sub-{ID}_task-{task}/preproc1.fif', overwrite=True)



# Epoch

In [ ]:
#epochs #number of epochs for participants should have a normal distribution
epochs = mne.make_fixed_length_epochs(data_filter_bads_reref_brainonly.copy(), duration=10, reject_by_annotation=True)


In [ ]:
##### Reject epochs with amplitude bigger than 2000 µVolt
#Peak to peak amplitude on brain scalp > 2000 µVolt are epochs not linked with physiological causes, physiological amplitude accepted < 800 µVolt
epochs_clean = epochs.copy()#.load_data()
epochs_clean.drop_bad({'eeg':2000*1e-6})
epochs_clean.plot_drop_log()
epochs_clean

# ICA

In [ ]:
## ##### Manual selection of ICA components
#Number of components = 30 because it's better to have as many components as possible (up to the nb of electrodes ~100 after clearing and rejection of non brain) 30 remains a good compromise
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
ica = ICA(n_components=30, max_iter='auto', random_state=97)
ica.fit(epochs_clean)
ica.plot_sources(epochs_clean)
ica.plot_components(inst=epochs_clean)

In [ ]:
#Main artifacts, eyes and jaw muscle contractions and clear non physiological artifacts (machine at the ICU for example). Coordinates in the variance table, x corresponds to epochs, y corresponds to variance. Enables to identify if the component has an impact on the variance all recording long or rather if it is a specific epoch that we need to reject. we need to decide if it is a component to remove or rather an epoch.
ica.plot_properties(epochs_clean, picks=ica.exclude)
ica

In [ ]:
#Double check which component to remove:
ica.exclude
len(ica.exclude)

In [ ]:
##### Remove component definetely
epochs_clean.load_data()
eeg_postica= ica.apply(epochs_clean.copy())

In [ ]:
#PLot to compare both signals pre and post ICA
epochs_clean.plot(title='raw', n_epochs=3, n_channels=60, scalings=20e-6)
eeg_postica.plot(title='ICA correction', n_epochs=3, n_channels=60, scalings=20e-6)


### Save individual files (Final)

In [ ]:
# #save final file
# if os.path.isdir('preproc_results')==False:
#     os.mkdir('preproc_results')
# if not os.path.exists(f'./preproc_results/sub-{ID}/sub-{ID}_task-{task}'):
#     os.makedirs(f'./preproc_results/sub-{ID}/sub-{ID}_task-{task}')
#     print('new directory made')
# data_filter_bads_reref_brainonly.save(f'./preproc_results/sub-{ID}/sub-{ID}_task-{task}/data_filter_bads_reref_brainonly_epoch_ICA.fif', overwrite=True)



## Save previously loaded figures

In [ ]:
if not os.path.exists(f'./{project}/out_figures/sub-{ID}/task-{task}'):
    os.makedirs(f'./{project}/out_figures/sub-{ID}/task-{task}')
    print('new directory made')

#data
step = 'data'
data.info['bads'].extend(['Vertex Reference'])
data.compute_psd(picks='eeg', fmax=70).plot()
data.info['bads'].remove('Vertex Reference')
data.info['bads']
if not os.path.exists(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png'):
    plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
    print(f'{step} PSD saved')
else:
    print(f'{step}.png already exists, Would you like to overwrite it?')
    yesorno = input('type "yes" to overwrite or "no" to cancel:')
    if yesorno == 'yes':
        plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
        print('PSD was overwritten')
    elif yesorno == 'no':
        print('Canceled')

#filter
step = 'data_filter'
data_filter.info['bads'].extend(['Vertex Reference'])
data_filter.compute_psd(picks='eeg', fmax=70).plot()
data_filter.info['bads'].remove('Vertex Reference')
if not os.path.exists(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png'):
    plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
    print(f'{step} PSD saved')
else:
    print(f'{step}.png already exists, Would you like to overwrite it?')
    yesorno = input('type "yes" to overwrite or "no" to cancel:')
    if yesorno == 'yes':
        plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
        print('PSD was overwritten')
    elif yesorno == 'no':
        print('Canceled')

#bads
step = 'data_filter_bads'
data_filter_bads.info['bads'].extend(['Vertex Reference'])
data_filter_bads.compute_psd(picks='eeg', fmax=70).plot()
data_filter_bads.info['bads'].remove('Vertex Reference')
data_filter_bads.info['bads']
if not os.path.exists(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png'):
    plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
    print(f'{step} PSD saved')
else:
    print(f'{step}.png already exists, Would you like to overwrite it?')
    yesorno = input('type "yes" to overwrite or "no" to cancel:')
    if yesorno == 'yes':
        plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
        print('PSD was overwritten')
    elif yesorno == 'no':
        print('Canceled')
if not os.path.exists(f'{new_path}/sub-{ID}_task-{task}_marked_bads.txt'):
    open(f'{new_path}/sub-{ID}_task-{task}_marked_bads.txt', 'x')
    with open (f'{new_path}/sub-{ID}_task-{task}_marked_bads.txt', 'w') as badsfile:
        badsfile.write('\n'.join(marked_bad))
        print('.txt file created')
else:
    with open (f'{new_path}/sub-{ID}_task-{task}_marked_bads.txt', 'w') as outfile:
        outfile.write('\n'.join(marked_bad))
        print('marked_bads.txt was updated')
        
#reref
step = 'data_filter_bads_reref'
data_filter_bads_reref.compute_psd(picks='eeg', fmax=70).plot()
if not os.path.exists(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png'):
    plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
    print(f'{step} PSD saved')
else:
    print(f'{step}.png already exists, Would you like to overwrite it?')
    yesorno = input('type "yes" to overwrite or "no" to cancel:')
    if yesorno == 'yes':
        plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
        print('PSD was overwritten')
    elif yesorno == 'no':
        print('Canceled')

#brainonly
step = 'data_filter_bads_reref_brainonly'
data_filter_bads_reref_brainonly.compute_psd(picks='eeg', fmax=70).plot()
if not os.path.exists(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png'):
    plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
    print(f'{step} PSD saved')
else:
    print(f'{step}.png already exists, Would you like to overwrite it?')
    yesorno = input('type "yes" to overwrite or "no" to cancel:')
    if yesorno == 'yes':
        plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
        print('PSD was overwritten')
    elif yesorno == 'no':
        print('Canceled')

#interpolate
step = 'data_filter_bads_reref_brainonly_interpolate'
data_filter_bads_reref_brainonly_interpolate.compute_psd(picks='eeg', fmax=70).plot()
if not os.path.exists(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png'):
    plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
    print(f'{step} PSD saved')
else:
    print(f'{step}.png already exists, Would you like to overwrite it?')
    yesorno = input('type "yes" to overwrite or "no" to cancel:')
    if yesorno == 'yes':
        plt.savefig(f'./{project}/out_figures/sub-{ID}/task-{task}/sub-{ID}_task-{task}_PSD_{step}.png')
        print('PSD was overwritten')
    elif yesorno == 'no':
        print('Canceled')